In [2]:
import google.generativeai as genai 
import langchain
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.llms import GooglePalm
from langchain_google_genai import GoogleGenerativeAIEmbeddings


d:\LLM_Langchain_Pinecone\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
langchain.__version__

'0.1.0'

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
#Let's read the document
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents= file_loader.load()
    return documents


In [6]:
doc = read_doc('./documents/')

In [7]:
len(doc)

39

In [8]:
# divide the document into chunks

def chunk_data(docs, chunk_size= 800, chunk_ovelap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size= chunk_size)
    doc=text_splitter.split_documents(docs)
    return doc



In [9]:
documents = chunk_data(docs=doc)
documents

[Document(page_content='Intelligence  partner  2022  \n \nSECTOR REPORT 2022  \nAEROSPACE  & DEFENCE', metadata={'source': 'documents\\aerospace-defense-sector-risk-report-2022.pdf', 'page': 0}),
 Document(page_content='2 | Aerospace & Defence Risk  Report  2022  \n2022', metadata={'source': 'documents\\aerospace-defense-sector-risk-report-2022.pdf', 'page': 1}),
 Document(page_content='3 | Aerospace & Defence Risk  Report  2022  \n2022   \n \nTable of Contents  \nPreface  ................................ ................................ ................................ ................................ ..........................  4 \nExecutive Summary  ................................ ................................ ................................ ................................ .......  5 \n1. Introduction  ................................ ................................ ................................ ................................ ..........  6 \n2. Bottom -Up Risk Assessment

In [45]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")


In [46]:
vector[:5]

[0.05636945, 0.0048285457, -0.0762591, -0.023642512, 0.05329321]

In [47]:
len(vector)
import os


In [48]:
pinecone.init(
    api_key=os.environ['PINECONE_API'],
    environment="gcp-starter"
)
index_name="langchain"

In [49]:
index =Pinecone.from_documents(doc, embeddings,index_name=index_name)

In [50]:
## Cosine Similarity Retrieve Result from Vector DB
def retrieve_query(query, k=2):
    matching_results = index.similarity_search(query, k=k)
    return matching_results

In [51]:
from langchain.chains.question_answering import load_qa_chain
from langchain_google_genai import GoogleGenerativeAI

#from langchain_community.llms import VertexAI


In [52]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat = ChatGoogleGenerativeAI(model="gemini-pro")


In [53]:
llm = GoogleGenerativeAI(model="gemini-pro")


In [54]:
chain = load_qa_chain(llm, chain_type="stuff")

In [55]:
### search answers from VectorDB
def retrieve_answers(query):
    doc_search = retrieve_query(query)
    print(doc_search)
    response = chain.run(input_documents= doc_search, question= query)
    return response



In [59]:
ad_query ="What is report based on"
answer = retrieve_answers(ad_query)
print(answer)

[Document(page_content='37 | Aerospace & Defence Risk  Report  2022  \n2022   \n \n \n \n \n \n  ', metadata={'page': 36.0, 'source': 'documents\\aerospace-defense-sector-risk-report-2022.pdf'}), Document(page_content='37 | Aerospace & Defence Risk  Report  2022  \n2022   \n \n \n \n \n \n  ', metadata={'page': 36.0, 'source': 'documents\\aerospace-defense-sector-risk-report-2022.pdf'})]


d:\LLM_Langchain_Pinecone\venv\lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
d:\LLM_Langchain_Pinecone\venv\lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
d:\LLM_Langchain_Pinecone\venv\lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


I cannot answer this question since the provided text does not contain information about what the report is based on.
